Make DataSets

1. All features + Reduced Embeddings 6 [done]
2. All features + Reduced Embeddings 8 [done]
3. All features + AllEmbeddings [done]
4. GE(10) + W2V(6) [done]
5. GE(10) + W2V(8) [done]
6. GE(10) + AllEmbeddings [done]
7. GE(100) + W2V(6) [done]
8. GE(100) + W2V(8) [done]
9. GE(100) + AllEmbeddings [done]

In [85]:
import os
import pandas as pd
import time
import numpy as np
from matplotlib import pyplot as plt
import itertools
import numpy as np
from scipy import stats
import pylab as pl
from sklearn import svm, linear_model
from sklearn.model_selection import cross_validate
from collections import OrderedDict
from scipy import linalg
from sklearn.preprocessing import StandardScaler

In [117]:
pickle_path="D:\\Courses\\InfoStorage\\Project\\Data\\FE"
item_stats= pd.read_pickle(pickle_path+"\\FeaturePickle\\product_stats.pkl")
queries_input= pd.read_pickle(pickle_path+"\\FeaturePickle\\queries_input.pkl")
query_item = pd.read_pickle(pickle_path+"\\FeaturePickle\\query_item.pkl")
item_graphEmbedding10 = pd.read_pickle(pickle_path+"\\item_GraphEmbeddings10.pkl")
item_graphEmbedding100 = pd.read_pickle(pickle_path+"\\item_GraphEmbeddings100.pkl")
vector_gensimProduct6 = pd.read_pickle(pickle_path+"\\Vector_gensim_product_6.pkl")
vector_gensimProduct8 = pd.read_pickle(pickle_path+"\\Vector_gensim_product_8.pkl")
vector_gensimQuery6 = pd.read_pickle(pickle_path+"\\Vector_gensim_query_6.pkl")
vector_gensimQuery8 = pd.read_pickle(pickle_path+"\\Vector_gensim_query_8.pkl")

In [118]:
item_stats.fillna(value=0, inplace=True)
X = pd.merge(query_item,item_stats,on='itemId',how='left')
X.head()

,queryId,itemId,vector_gensim_query,Relevance,is.test,item_show_count,Item_clicks,Item_view_count,Item_purchase_count,userShow,userView,userPurchase,CTR,View Rate,Click Value Rate,vector_gensim_product
0,1,7518,"[-0.006535598, 0.0024064586, 0.0074732024, -0....",0,False,148.0,70.0,57.0,0.0,47.0,3.0,1.0,0.472973,0.385135,0.000000,"[0.009888342, 0.006329806, 0.0005410195, -0.00..."
1,1,71,"[-0.006535598, 0.0024064586, 0.0074732024, -0....",0,False,192.0,92.0,73.0,2.0,102.0,2.0,1.0,0.479167,0.380208,0.010417,"[-0.0083958395, -0.0075500184, 0.0025241065, 0..."
2,1,30311,"[-0.006535598, 0.0024064586, 0.0074732024, -0....",0,False,108.0,164.0,116.0,2.0,37.0,2.0,1.0,1.518519,1.074074,0.018519,"[0.007938938, 0.005018753, 0.0023497294, 0.001..."
3,1,7837,"[-0.006535598, 0.0024064586, 0.0074732024, -0....",0,False,281.0,220.0,191.0,3.0,82.0,5.0,1.0,0.782918,0.679715,0.010676,"[-0.0075791064, -0.005229415, 0.008466356, -0...."
4,1,30792,"[-0.006535598, 0.0024064586, 0.0074732024, -0....",0,False,389.0,475.0,445.0,11.0,143.0,11.0,1.0,1.221080,1.143959,0.028278,"[-0.0057816627, 0.009380997, 0.00030535308, -0..."


# GraphEmbedding10 + W2V(6)

In [119]:
item_graphEmbedding100.head()
c=[]
for i in range(0,16):
    c.append("graphfeature"+str(i))
c.append("itemId")
item_graphEmbedding100.columns=c
item_graphEmbedding100.head()

,graphfeature0,graphfeature1,graphfeature2,graphfeature3,graphfeature4,graphfeature5,graphfeature6,graphfeature7,graphfeature8,graphfeature9,graphfeature10,graphfeature11,graphfeature12,graphfeature13,graphfeature14,graphfeature15,itemId
0,-0.104883,-0.259306,1.515564,0.875736,-0.593640,0.517496,2.190138,-1.117627,-1.774247,2.020438,-1.478869,1.398075,1.266815,1.739026,-1.630902,-1.230166,2
1,-0.131535,-0.206588,1.414965,0.831427,-0.545699,0.523763,2.041376,-1.017171,-1.621164,1.878158,-1.370647,1.268919,1.214066,1.630000,-1.504339,-1.098514,6
2,-0.027140,-0.350058,1.510682,0.834832,-0.627208,0.431065,2.160386,-1.179683,-1.863743,2.012402,-1.489170,1.475488,1.202769,1.710547,-1.663073,-1.331465,7
3,-0.128081,-0.229844,1.471010,0.862143,-0.552211,0.510502,2.142078,-1.079301,-1.713666,1.990805,-1.456616,1.347056,1.226706,1.709289,-1.594422,-1.185544,9
4,-0.099088,-0.092104,0.786219,0.471612,-0.269811,0.286465,1.152249,-0.567173,-0.902310,1.088997,-0.803421,0.710483,0.661084,0.925238,-0.854593,-0.610051,10


In [120]:
X = X.drop([ 'vector_gensim_query', 'vector_gensim_product', \
           'item_show_count','Item_clicks','Item_view_count','Item_purchase_count','userShow','userView',\
            'userPurchase','CTR','View Rate','Click Value Rate'],axis=1)
X.head()
#X = pd.merge(X,vector_gensimProduct6,on='itemId',how='left')
#X = pd.merge(X,vector_gensimQuery6,on='queryId',how='left')
X = pd.merge(X,vector_gensimProduct8,on='itemId',how='left')
X = pd.merge(X,vector_gensimQuery8,on='queryId',how='left')
X = pd.merge(X,item_graphEmbedding100,on='itemId',how='left')
X.head()
Train = X[X['is.test']==False]
Test = X[X['is.test']== True]
Train= Train.drop(['is.test'], axis=1)
Test= Test.drop(['is.test'], axis=1)

In [122]:
Train['vector_gensim_product'][0]

array([-0.0145327 ,  0.04172187,  0.03485874,  0.01078536,  0.01587863,
       -0.02414756, -0.05358342,  0.03255361], dtype=float32)

In [123]:
test=[]
query_item_order_test=[]
labels_test=[]
for index,row in Test.iterrows():
    r=[]
    query_item_order_test.append([row['queryId'],row['itemId']])
    
    r.append(row['graphfeature0'])
    r.append(row['graphfeature1'])
    r.append(row['graphfeature2'])
    r.append(row['graphfeature3'])
    r.append(row['graphfeature4'])
    r.append(row['graphfeature5'])
    r.append(row['graphfeature6'])
    r.append(row['graphfeature7'])
    r.append(row['graphfeature8'])
    r.append(row['graphfeature9'])
    r.append(row['graphfeature10'])
    r.append(row['graphfeature11'])
    r.append(row['graphfeature12'])
    r.append(row['graphfeature13'])
    r.append(row['graphfeature14'])
    r.append(row['graphfeature15'])
    r= r + row['vector_gensim_query'].tolist()
    r= r + row['vector_gensim_product'].tolist()
    
    test.append(r)
    labels_test.append(row['Relevance'])

In [124]:
train=[]
query_item_order=[]
labels=[]
for index,row in Train.iterrows():
    r=[]
    query_item_order.append([row['queryId'],row['itemId']])
    
    r.append(row['graphfeature0'])
    r.append(row['graphfeature1'])
    r.append(row['graphfeature2'])
    r.append(row['graphfeature3'])
    r.append(row['graphfeature4'])
    r.append(row['graphfeature5'])
    r.append(row['graphfeature6'])
    r.append(row['graphfeature7'])
    r.append(row['graphfeature8'])
    r.append(row['graphfeature9'])
    r.append(row['graphfeature10'])
    r.append(row['graphfeature11'])
    r.append(row['graphfeature12'])
    r.append(row['graphfeature13'])
    r.append(row['graphfeature14'])
    r.append(row['graphfeature15'])
    r= r + row['vector_gensim_query'].tolist()
    r= r + row['vector_gensim_product'].tolist()
    
    train.append(r)
    labels.append(row['Relevance'])

In [125]:
train = np.array(train)
test= np.array(test)
sc = StandardScaler()
train = sc.fit_transform(train)
test = sc.transform(test)
L=Train['queryId'].value_counts(sort=True, ascending=True)
L = dict(L)
L= OrderedDict(sorted(L.items()))
print(len(L))
print(len(labels))

35615
703519


In [126]:
#http://fa.bianp.net/blog/2012/learning-to-rank-with-scikit-learn-the-pairwise-transform/
k = 0
Xp, yp, diff = [], [], []
s=0
e=0
theta = np.deg2rad(60)
w = np.array([np.sin(theta), np.cos(theta)]) #for plotting
for key in L:
    comb = itertools.combinations(range(L.get(key)), 2)
    #create the first batch
    s=e
    e=s+L.get(key) -1
    #print("Start:%d, End %d" %(s,e))
    for (i, j) in comb:
        i=i+s
        j=j+e-1
        if labels[i] == labels[j] or query_item_order[i][0] != query_item_order[j][0] :
            # skip if same target or different group(different qIds)
            continue
        Xp.append(train[i] - train[j])
        diff.append(labels[i] - labels[j])
        yp.append(np.sign(diff[-1]))
        # output balanced classes
        if yp[-1] != (-1) ** k:
            yp[-1] *= -1
            Xp[-1] *= -1
            diff[-1] *= -1
        k += 1
Xp, yp, diff = map(np.asanyarray, (Xp, yp, diff))

In [127]:
qio = np.array(query_item_order)
lbls = np.array(labels)
qio_test = np.array(query_item_order_test)
lbls_test = np.array(labels_test)
# np.loadtxt np.savetxt
path_to_save="D:\\Courses\\InfoStorage\\Project\\Data\\Numpy_Objects\\"
X.to_pickle(path_to_save+"AllFeaturesCombined.pkl")
Train.to_pickle(path_to_save+"Train.pkl")
Test.to_pickle(path_to_save+"Test.pkl")
np.savetxt(path_to_save+'GraphFeatures100WithWord2VecEmbeddings8_train.txt',train,fmt='%1.15f')
np.savetxt(path_to_save+'GraphFeatures100WithWord2VecEmbeddingsAfterPairWise8_Xp.txt',Xp,fmt='%1.15f')
np.savetxt(path_to_save+'GraphFeatures100WithWord2VecEmbeddingsAfterPairWise8_yp.txt',yp,fmt='%1.15f')
np.savetxt(path_to_save+'GraphFeatures100WithWord2VecEmbeddings8_test.txt',test,fmt='%1.15f')
np.savetxt(path_to_save+'GraphFeatures100WithWord2VecEmbeddings8_query_item_order.txt',qio,fmt='%1.15f')
np.savetxt(path_to_save+'GraphFeatures100WithWord2VecEmbeddings8_labels.txt',lbls,fmt='%1.15f')
np.savetxt(path_to_save+'GraphFeatures100WithWord2VecEmbeddings8_query_item_order_test.txt',qio_test,fmt='%1.15f')
np.savetxt(path_to_save+'GraphFeatures100WithWord2VecEmbeddings8_labels_test.txt',lbls_test,fmt='%1.15f')

# All Features + All Embeddings

In [39]:
Train = X[X['is.test']==False]
Test = X[X['is.test']== True]
Train= Train.drop(['is.test'], axis=1)
Test= Test.drop(['is.test'], axis=1)

In [40]:
test=[]
query_item_order_test=[]
labels_test=[]
for index,row in Test.iterrows():
    r=[]
    query_item_order_test.append([row['queryId'],row['itemId']])
    
    r.append(row['item_show_count'])
    r.append(row['Item_clicks'])
    r.append(row['Item_view_count'])
    r.append(row['Item_purchase_count'])
    r.append(row['CTR'])
    r.append(row['userShow'])
    r.append(row['userView'])
    r.append(row['userPurchase'])
    r.append(row['View Rate'])
    r.append(row['Click Value Rate'])
    r= r + row['vector_gensim_query'].tolist()
    r= r + row['vector_gensim_product'].tolist()
    
    test.append(r)
    labels_test.append(row['Relevance'])

In [41]:
train=[]
query_item_order=[]
labels=[]
for index,row in Train.iterrows():
    r=[]
    query_item_order.append([row['queryId'],row['itemId']])
    
    r.append(row['item_show_count'])
    r.append(row['Item_clicks'])
    r.append(row['Item_view_count'])
    r.append(row['Item_purchase_count'])
    r.append(row['CTR'])
    r.append(row['userShow'])
    r.append(row['userView'])
    r.append(row['userPurchase'])
    r.append(row['View Rate'])
    r.append(row['Click Value Rate'])
    r= r + row['vector_gensim_query'].tolist()
    r= r + row['vector_gensim_product'].tolist()
    
    train.append(r)
    labels.append(row['Relevance'])

In [42]:
train = np.array(train)
test= np.array(test)
sc = StandardScaler()
train = sc.fit_transform(train)
test = sc.transform(test)
L=Train['queryId'].value_counts(sort=True, ascending=True)
L = dict(L)
L= OrderedDict(sorted(L.items()))
print(len(L))
print(len(labels))

35615
703519


In [43]:
train.shape

(703519, 110)

In [44]:
#http://fa.bianp.net/blog/2012/learning-to-rank-with-scikit-learn-the-pairwise-transform/
k = 0
Xp, yp, diff = [], [], []
s=0
e=0
theta = np.deg2rad(60)
w = np.array([np.sin(theta), np.cos(theta)]) #for plotting
for key in L:
    comb = itertools.combinations(range(L.get(key)), 2)
    #create the first batch
    s=e
    e=s+L.get(key) -1
    #print("Start:%d, End %d" %(s,e))
    for (i, j) in comb:
        i=i+s
        j=j+e-1
        if labels[i] == labels[j] or query_item_order[i][0] != query_item_order[j][0] :
            # skip if same target or different group(different qIds)
            continue
        Xp.append(train[i] - train[j])
        diff.append(labels[i] - labels[j])
        yp.append(np.sign(diff[-1]))
        # output balanced classes
        if yp[-1] != (-1) ** k:
            yp[-1] *= -1
            Xp[-1] *= -1
            diff[-1] *= -1
        k += 1
Xp, yp, diff = map(np.asanyarray, (Xp, yp, diff))

Start:0, End 19
Start:19, End 40
Start:40, End 59
Start:59, End 78
Start:78, End 96
Start:96, End 118
Start:118, End 136
Start:136, End 155
Start:155, End 174
Start:174, End 193
Start:193, End 212
Start:212, End 231
Start:231, End 250
Start:250, End 271
Start:271, End 289
Start:289, End 309
Start:309, End 330
Start:330, End 352
Start:352, End 371
Start:371, End 390
Start:390, End 409
Start:409, End 432
Start:432, End 451
Start:451, End 470
Start:470, End 489
Start:489, End 508
Start:508, End 527
Start:527, End 546
Start:546, End 565
Start:565, End 584
Start:584, End 607
Start:607, End 625
Start:625, End 646
Start:646, End 665
Start:665, End 684
Start:684, End 703
Start:703, End 722
Start:722, End 742
Start:742, End 762
Start:762, End 780
Start:780, End 801
Start:801, End 815
Start:815, End 834
Start:834, End 854
Start:854, End 872
Start:872, End 891
Start:891, End 913
Start:913, End 932
Start:932, End 952
Start:952, End 970
Start:970, End 989
Start:989, End 1008
Start:1008, End 1027
St

Start:16487, End 16509
Start:16509, End 16528
Start:16528, End 16547
Start:16547, End 16566
Start:16566, End 16585
Start:16585, End 16604
Start:16604, End 16623
Start:16623, End 16645
Start:16645, End 16661
Start:16661, End 16680
Start:16680, End 16699
Start:16699, End 16718
Start:16718, End 16738
Start:16738, End 16758
Start:16758, End 16777
Start:16777, End 16796
Start:16796, End 16815
Start:16815, End 16834
Start:16834, End 16853
Start:16853, End 16873
Start:16873, End 16893
Start:16893, End 16913
Start:16913, End 16933
Start:16933, End 16952
Start:16952, End 16971
Start:16971, End 16990
Start:16990, End 17001
Start:17001, End 17020
Start:17020, End 17039
Start:17039, End 17058
Start:17058, End 17081
Start:17081, End 17100
Start:17100, End 17125
Start:17125, End 17150
Start:17150, End 17169
Start:17169, End 17188
Start:17188, End 17207
Start:17207, End 17226
Start:17226, End 17241
Start:17241, End 17260
Start:17260, End 17279
Start:17279, End 17290
Start:17290, End 17311
Start:17311

Start:29718, End 29739
Start:29739, End 29757
Start:29757, End 29778
Start:29778, End 29797
Start:29797, End 29816
Start:29816, End 29835
Start:29835, End 29847
Start:29847, End 29866
Start:29866, End 29886
Start:29886, End 29905
Start:29905, End 29914
Start:29914, End 29932
Start:29932, End 29952
Start:29952, End 29972
Start:29972, End 29983
Start:29983, End 30010
Start:30010, End 30029
Start:30029, End 30038
Start:30038, End 30057
Start:30057, End 30077
Start:30077, End 30092
Start:30092, End 30111
Start:30111, End 30130
Start:30130, End 30157
Start:30157, End 30176
Start:30176, End 30196
Start:30196, End 30215
Start:30215, End 30234
Start:30234, End 30247
Start:30247, End 30266
Start:30266, End 30285
Start:30285, End 30299
Start:30299, End 30319
Start:30319, End 30339
Start:30339, End 30358
Start:30358, End 30377
Start:30377, End 30396
Start:30396, End 30407
Start:30407, End 30426
Start:30426, End 30437
Start:30437, End 30456
Start:30456, End 30476
Start:30476, End 30495
Start:30495

Start:41738, End 41757
Start:41757, End 41776
Start:41776, End 41798
Start:41798, End 41819
Start:41819, End 41839
Start:41839, End 41858
Start:41858, End 41877
Start:41877, End 41898
Start:41898, End 41919
Start:41919, End 41939
Start:41939, End 41959
Start:41959, End 41978
Start:41978, End 41997
Start:41997, End 42017
Start:42017, End 42040
Start:42040, End 42059
Start:42059, End 42079
Start:42079, End 42098
Start:42098, End 42108
Start:42108, End 42127
Start:42127, End 42146
Start:42146, End 42165
Start:42165, End 42184
Start:42184, End 42203
Start:42203, End 42223
Start:42223, End 42242
Start:42242, End 42261
Start:42261, End 42281
Start:42281, End 42300
Start:42300, End 42319
Start:42319, End 42338
Start:42338, End 42358
Start:42358, End 42379
Start:42379, End 42400
Start:42400, End 42419
Start:42419, End 42438
Start:42438, End 42457
Start:42457, End 42476
Start:42476, End 42496
Start:42496, End 42515
Start:42515, End 42534
Start:42534, End 42549
Start:42549, End 42568
Start:42568

Start:53908, End 53927
Start:53927, End 53946
Start:53946, End 53964
Start:53964, End 53982
Start:53982, End 54004
Start:54004, End 54023
Start:54023, End 54045
Start:54045, End 54064
Start:54064, End 54083
Start:54083, End 54102
Start:54102, End 54121
Start:54121, End 54145
Start:54145, End 54164
Start:54164, End 54183
Start:54183, End 54202
Start:54202, End 54221
Start:54221, End 54240
Start:54240, End 54258
Start:54258, End 54277
Start:54277, End 54296
Start:54296, End 54315
Start:54315, End 54334
Start:54334, End 54353
Start:54353, End 54372
Start:54372, End 54392
Start:54392, End 54411
Start:54411, End 54430
Start:54430, End 54450
Start:54450, End 54470
Start:54470, End 54492
Start:54492, End 54512
Start:54512, End 54531
Start:54531, End 54545
Start:54545, End 54564
Start:54564, End 54583
Start:54583, End 54603
Start:54603, End 54622
Start:54622, End 54637
Start:54637, End 54656
Start:54656, End 54673
Start:54673, End 54698
Start:54698, End 54717
Start:54717, End 54736
Start:54736

...

Start:597792, End 597811
Start:597811, End 597832
Start:597832, End 597853
Start:597853, End 597871
Start:597871, End 597890
Start:597890, End 597909
Start:597909, End 597928
Start:597928, End 597948
Start:597948, End 597967
Start:597967, End 597985
Start:597985, End 598007
Start:598007, End 598026
Start:598026, End 598042
Start:598042, End 598060
Start:598060, End 598079
Start:598079, End 598098
Start:598098, End 598117
Start:598117, End 598136
Start:598136, End 598159
Start:598159, End 598179
Start:598179, End 598197
Start:598197, End 598216
Start:598216, End 598236
Start:598236, End 598255
Start:598255, End 598272
Start:598272, End 598291
Start:598291, End 598308
Start:598308, End 598327
Start:598327, End 598346
Start:598346, End 598368
Start:598368, End 598392
Start:598392, End 598413
Start:598413, End 598432
Start:598432, End 598451
Start:598451, End 598472
Start:598472, End 598492
Start:598492, End 598511
Start:598511, End 598523
Start:598523, End 598542
Start:598542, End 598561


Start:616554, End 616573
Start:616573, End 616596
Start:616596, End 616615
Start:616615, End 616634
Start:616634, End 616653
Start:616653, End 616672
Start:616672, End 616691
Start:616691, End 616710
Start:616710, End 616722
Start:616722, End 616741
Start:616741, End 616759
Start:616759, End 616778
Start:616778, End 616793
Start:616793, End 616812
Start:616812, End 616832
Start:616832, End 616850
Start:616850, End 616869
Start:616869, End 616888
Start:616888, End 616909
Start:616909, End 616929
Start:616929, End 616948
Start:616948, End 616968
Start:616968, End 616988
Start:616988, End 617009
Start:617009, End 617027
Start:617027, End 617046
Start:617046, End 617063
Start:617063, End 617082
Start:617082, End 617101
Start:617101, End 617125
Start:617125, End 617144
Start:617144, End 617168
Start:617168, End 617186
Start:617186, End 617205
Start:617205, End 617224
Start:617224, End 617243
Start:617243, End 617263
Start:617263, End 617282
Start:617282, End 617301
Start:617301, End 617320


Start:633982, End 634001
Start:634001, End 634020
Start:634020, End 634039
Start:634039, End 634057
Start:634057, End 634070
Start:634070, End 634089
Start:634089, End 634107
Start:634107, End 634126
Start:634126, End 634143
Start:634143, End 634162
Start:634162, End 634180
Start:634180, End 634192
Start:634192, End 634212
Start:634212, End 634231
Start:634231, End 634250
Start:634250, End 634269
Start:634269, End 634288
Start:634288, End 634302
Start:634302, End 634321
Start:634321, End 634340
Start:634340, End 634359
Start:634359, End 634378
Start:634378, End 634388
Start:634388, End 634407
Start:634407, End 634426
Start:634426, End 634445
Start:634445, End 634464
Start:634464, End 634483
Start:634483, End 634503
Start:634503, End 634522
Start:634522, End 634541
Start:634541, End 634560
Start:634560, End 634580
Start:634580, End 634602
Start:634602, End 634620
Start:634620, End 634640
Start:634640, End 634659
Start:634659, End 634679
Start:634679, End 634697
Start:634697, End 634716


Start:644511, End 644530
Start:644530, End 644549
Start:644549, End 644568
Start:644568, End 644590
Start:644590, End 644609
Start:644609, End 644629
Start:644629, End 644648
Start:644648, End 644668
Start:644668, End 644687
Start:644687, End 644700
Start:644700, End 644718
Start:644718, End 644738
Start:644738, End 644756
Start:644756, End 644775
Start:644775, End 644794
Start:644794, End 644814
Start:644814, End 644833
Start:644833, End 644853
Start:644853, End 644872
Start:644872, End 644891
Start:644891, End 644910
Start:644910, End 644930
Start:644930, End 644939
Start:644939, End 644959
Start:644959, End 644977
Start:644977, End 644995
Start:644995, End 645014
Start:645014, End 645033
Start:645033, End 645050
Start:645050, End 645069
Start:645069, End 645087
Start:645087, End 645106
Start:645106, End 645126
Start:645126, End 645145
Start:645145, End 645164
Start:645164, End 645183
Start:645183, End 645201
Start:645201, End 645220
Start:645220, End 645235
Start:645235, End 645252


Start:660586, End 660604
Start:660604, End 660620
Start:660620, End 660637
Start:660637, End 660656
Start:660656, End 660675
Start:660675, End 660686
Start:660686, End 660705
Start:660705, End 660724
Start:660724, End 660743
Start:660743, End 660763
Start:660763, End 660782
Start:660782, End 660792
Start:660792, End 660809
Start:660809, End 660828
Start:660828, End 660846
Start:660846, End 660865
Start:660865, End 660886
Start:660886, End 660905
Start:660905, End 660924
Start:660924, End 660943
Start:660943, End 660962
Start:660962, End 660982
Start:660982, End 661001
Start:661001, End 661019
Start:661019, End 661037
Start:661037, End 661055
Start:661055, End 661074
Start:661074, End 661093
Start:661093, End 661112
Start:661112, End 661132
Start:661132, End 661151
Start:661151, End 661169
Start:661169, End 661188
Start:661188, End 661207
Start:661207, End 661227
Start:661227, End 661245
Start:661245, End 661263
Start:661263, End 661282
Start:661282, End 661301
Start:661301, End 661320


In [46]:
qio = np.array(query_item_order)
lbls = np.array(labels)
qio_test = np.array(query_item_order_test)
lbls_test = np.array(labels_test)
# np.loadtxt np.savetxt
path_to_save="D:\\Courses\\InfoStorage\\Project\\Data\\Numpy_Objects\\"
X.to_pickle(path_to_save+"AllFeaturesCombined.pkl")
Train.to_pickle(path_to_save+"Train.pkl")
Test.to_pickle(path_to_save+"Test.pkl")
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsall_train.txt',train,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsAfterPairWiseall_Xp.txt',Xp,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsAfterPairWiseall_yp.txt',yp,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsall_test.txt',test,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsall_query_item_order.txt',qio,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsall_labels.txt',lbls,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsall_query_item_order_test.txt',qio_test,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsall_labels_test.txt',lbls_test,fmt='%1.15f')

# All Features + Reduced Embeddings 8

In [27]:
vector_gensimProduct8.head()

,itemId,vector_gensim_product
0,1,"[0.020159593, -0.058558576, -0.01950021, -0.01..."
1,2,"[0.05712696, -0.046231538, -0.04982928, -0.042..."
2,3,"[0.019510211, -0.014173324, 0.042070888, 0.056..."
3,4,"[-0.046358127, 0.035930954, -0.015230325, -0.0..."
4,6,"[0.054646514, 0.037913945, -0.011987117, -0.02..."


In [28]:
vector_gensimQuery8.head()

,queryId,vector_gensim_query
0,1,"[0.002623201, 0.034680568, 0.03499031, 0.05087..."
1,2,"[0.047217622, -0.043985248, 0.057724018, -0.04..."
2,3,"[0.010164494, 0.020062938, 0.053123783, 0.0513..."
3,4,"[0.029296976, -0.04737446, 0.053250246, 0.0366..."
4,5,"[-0.0015976363, -0.017315872, 0.046792414, -0...."


In [29]:
X = X.drop(['vector_gensim_query','vector_gensim_product'],axis=1)
X.head()

,queryId,itemId,Relevance,is.test,item_show_count,Item_clicks,Item_view_count,Item_purchase_count,userShow,userView,userPurchase,CTR,View Rate,Click Value Rate
0,1,7518,0,False,148.0,70.0,57.0,0.0,47.0,3.0,1.0,0.472973,0.385135,0.000000
1,1,71,0,False,192.0,92.0,73.0,2.0,102.0,2.0,1.0,0.479167,0.380208,0.010417
2,1,30311,0,False,108.0,164.0,116.0,2.0,37.0,2.0,1.0,1.518519,1.074074,0.018519
3,1,7837,0,False,281.0,220.0,191.0,3.0,82.0,5.0,1.0,0.782918,0.679715,0.010676
4,1,30792,0,False,389.0,475.0,445.0,11.0,143.0,11.0,1.0,1.221080,1.143959,0.028278


In [30]:
X = pd.merge(X,vector_gensimProduct6,on='itemId',how='left')
X = pd.merge(X,vector_gensimQuery6,on='queryId',how='left')
X.head()
Train = X[X['is.test']==False]
Test = X[X['is.test']== True]
Train= Train.drop(['is.test'], axis=1)
Test= Test.drop(['is.test'], axis=1)

In [31]:
test=[]
query_item_order_test=[]
labels_test=[]
for index,row in Test.iterrows():
    r=[]
    query_item_order_test.append([row['queryId'],row['itemId']])
    
    r.append(row['item_show_count'])
    r.append(row['Item_clicks'])
    r.append(row['Item_view_count'])
    r.append(row['Item_purchase_count'])
    r.append(row['CTR'])
    r.append(row['userShow'])
    r.append(row['userView'])
    r.append(row['userPurchase'])
    r.append(row['View Rate'])
    r.append(row['Click Value Rate'])
    r= r + row['vector_gensim_query'].tolist()
    r= r + row['vector_gensim_product'].tolist()
    
    test.append(r)
    labels_test.append(row['Relevance'])

In [32]:
train=[]
query_item_order=[]
labels=[]
for index,row in Train.iterrows():
    r=[]
    query_item_order.append([row['queryId'],row['itemId']])
    
    r.append(row['item_show_count'])
    r.append(row['Item_clicks'])
    r.append(row['Item_view_count'])
    r.append(row['Item_purchase_count'])
    r.append(row['CTR'])
    r.append(row['userShow'])
    r.append(row['userView'])
    r.append(row['userPurchase'])
    r.append(row['View Rate'])
    r.append(row['Click Value Rate'])
    r= r + row['vector_gensim_query'].tolist()
    r= r + row['vector_gensim_product'].tolist()
    
    train.append(r)
    labels.append(row['Relevance'])

In [33]:
train = np.array(train)
test= np.array(test)
sc = StandardScaler()
train = sc.fit_transform(train)
test = sc.transform(test)
L=Train['queryId'].value_counts(sort=True, ascending=True)
L = dict(L)
L= OrderedDict(sorted(L.items()))
print(len(L))
print(len(labels))

35615
703519


In [34]:
#http://fa.bianp.net/blog/2012/learning-to-rank-with-scikit-learn-the-pairwise-transform/
k = 0
Xp, yp, diff = [], [], []
s=0
e=0
theta = np.deg2rad(60)
w = np.array([np.sin(theta), np.cos(theta)]) #for plotting
for key in L:
    comb = itertools.combinations(range(L.get(key)), 2)
    #create the first batch
    s=e
    e=s+L.get(key) -1
    print("Start:%d, End %d" %(s,e))
    for (i, j) in comb:
        i=i+s
        j=j+e-1
        if labels[i] == labels[j] or query_item_order[i][0] != query_item_order[j][0] :
            # skip if same target or different group(different qIds)
            continue
        Xp.append(train[i] - train[j])
        diff.append(labels[i] - labels[j])
        yp.append(np.sign(diff[-1]))
        # output balanced classes
        if yp[-1] != (-1) ** k:
            yp[-1] *= -1
            Xp[-1] *= -1
            diff[-1] *= -1
        k += 1
Xp, yp, diff = map(np.asanyarray, (Xp, yp, diff))

Start:0, End 19
Start:19, End 40
Start:40, End 59
Start:59, End 78
Start:78, End 96
Start:96, End 118
Start:118, End 136
Start:136, End 155
Start:155, End 174
Start:174, End 193
Start:193, End 212
Start:212, End 231
Start:231, End 250
Start:250, End 271
Start:271, End 289
Start:289, End 309
Start:309, End 330
Start:330, End 352
Start:352, End 371
Start:371, End 390
Start:390, End 409
Start:409, End 432
Start:432, End 451
Start:451, End 470
Start:470, End 489
Start:489, End 508
Start:508, End 527
Start:527, End 546
Start:546, End 565
Start:565, End 584
Start:584, End 607
Start:607, End 625
Start:625, End 646
Start:646, End 665
Start:665, End 684
Start:684, End 703
Start:703, End 722
Start:722, End 742
Start:742, End 762
Start:762, End 780
Start:780, End 801
Start:801, End 815
Start:815, End 834
Start:834, End 854
Start:854, End 872
Start:872, End 891
Start:891, End 913
Start:913, End 932
Start:932, End 952
Start:952, End 970
Start:970, End 989
Start:989, End 1008
Start:1008, End 1027
St

Start:14463, End 14487
Start:14487, End 14506
Start:14506, End 14529
Start:14529, End 14544
Start:14544, End 14563
Start:14563, End 14582
Start:14582, End 14601
Start:14601, End 14621
Start:14621, End 14642
Start:14642, End 14661
Start:14661, End 14681
Start:14681, End 14700
Start:14700, End 14718
Start:14718, End 14738
Start:14738, End 14757
Start:14757, End 14776
Start:14776, End 14796
Start:14796, End 14814
Start:14814, End 14834
Start:14834, End 14853
Start:14853, End 14872
Start:14872, End 14891
Start:14891, End 14910
Start:14910, End 14934
Start:14934, End 14948
Start:14948, End 14967
Start:14967, End 14986
Start:14986, End 15005
Start:15005, End 15025
Start:15025, End 15046
Start:15046, End 15065
Start:15065, End 15085
Start:15085, End 15104
Start:15104, End 15123
Start:15123, End 15138
Start:15138, End 15157
Start:15157, End 15177
Start:15177, End 15196
Start:15196, End 15215
Start:15215, End 15234
Start:15234, End 15253
Start:15253, End 15272
Start:15272, End 15291
Start:15291

Start:24903, End 24922
Start:24922, End 24942
Start:24942, End 24961
Start:24961, End 24980
Start:24980, End 25002
Start:25002, End 25021
Start:25021, End 25040
Start:25040, End 25059
Start:25059, End 25078
Start:25078, End 25097
Start:25097, End 25117
Start:25117, End 25137
Start:25137, End 25156
Start:25156, End 25175
Start:25175, End 25194
Start:25194, End 25213
Start:25213, End 25233
Start:25233, End 25252
Start:25252, End 25272
Start:25272, End 25291
Start:25291, End 25310
Start:25310, End 25329
Start:25329, End 25349
Start:25349, End 25372
Start:25372, End 25385
Start:25385, End 25404
Start:25404, End 25425
Start:25425, End 25444
Start:25444, End 25464
Start:25464, End 25483
Start:25483, End 25502
Start:25502, End 25527
Start:25527, End 25547
Start:25547, End 25564
Start:25564, End 25580
Start:25580, End 25599
Start:25599, End 25618
Start:25618, End 25637
Start:25637, End 25657
Start:25657, End 25676
Start:25676, End 25696
Start:25696, End 25715
Start:25715, End 25724
Start:25724

Start:38317, End 38336
Start:38336, End 38351
Start:38351, End 38371
Start:38371, End 38391
Start:38391, End 38410
Start:38410, End 38430
Start:38430, End 38449
Start:38449, End 38468
Start:38468, End 38488
Start:38488, End 38501
Start:38501, End 38520
Start:38520, End 38539
Start:38539, End 38559
Start:38559, End 38578
Start:38578, End 38597
Start:38597, End 38616
Start:38616, End 38635
Start:38635, End 38655
Start:38655, End 38674
Start:38674, End 38693
Start:38693, End 38712
Start:38712, End 38728
Start:38728, End 38746
Start:38746, End 38758
Start:38758, End 38777
Start:38777, End 38796
Start:38796, End 38809
Start:38809, End 38826
Start:38826, End 38846
Start:38846, End 38865
Start:38865, End 38884
Start:38884, End 38903
Start:38903, End 38922
Start:38922, End 38942
Start:38942, End 38961
Start:38961, End 38972
Start:38972, End 38991
Start:38991, End 39011
Start:39011, End 39030
Start:39030, End 39040
Start:39040, End 39059
Start:39059, End 39079
Start:39079, End 39098
Start:39098

Start:48486, End 48499
Start:48499, End 48518
Start:48518, End 48537
Start:48537, End 48548
Start:48548, End 48567
Start:48567, End 48586
Start:48586, End 48608
Start:48608, End 48627
Start:48627, End 48647
Start:48647, End 48666
Start:48666, End 48688
Start:48688, End 48708
Start:48708, End 48727
Start:48727, End 48739
Start:48739, End 48755
Start:48755, End 48772
Start:48772, End 48791
Start:48791, End 48810
Start:48810, End 48830
Start:48830, End 48849
Start:48849, End 48868
Start:48868, End 48887
Start:48887, End 48899
Start:48899, End 48918
Start:48918, End 48938
Start:48938, End 48957
Start:48957, End 48979
Start:48979, End 48998
Start:48998, End 49016
Start:49016, End 49035
Start:49035, End 49054
Start:49054, End 49073
Start:49073, End 49093
Start:49093, End 49113
Start:49113, End 49132
Start:49132, End 49142
Start:49142, End 49160
Start:49160, End 49179
Start:49179, End 49197
Start:49197, End 49217
Start:49217, End 49236
Start:49236, End 49255
Start:49255, End 49274
Start:49274

...

Start:601643, End 601665
Start:601665, End 601684
Start:601684, End 601703
Start:601703, End 601722
Start:601722, End 601741
Start:601741, End 601760
Start:601760, End 601781
Start:601781, End 601800
Start:601800, End 601819
Start:601819, End 601838
Start:601838, End 601854
Start:601854, End 601873
Start:601873, End 601892
Start:601892, End 601917
Start:601917, End 601938
Start:601938, End 601953
Start:601953, End 601974
Start:601974, End 601993
Start:601993, End 602012
Start:602012, End 602026
Start:602026, End 602045
Start:602045, End 602064
Start:602064, End 602082
Start:602082, End 602101
Start:602101, End 602121
Start:602121, End 602136
Start:602136, End 602155
Start:602155, End 602174
Start:602174, End 602193
Start:602193, End 602212
Start:602212, End 602231
Start:602231, End 602251
Start:602251, End 602270
Start:602270, End 602288
Start:602288, End 602304
Start:602304, End 602326
Start:602326, End 602343
Start:602343, End 602362
Start:602362, End 602381
Start:602381, End 602399


Start:618764, End 618784
Start:618784, End 618803
Start:618803, End 618823
Start:618823, End 618842
Start:618842, End 618861
Start:618861, End 618880
Start:618880, End 618893
Start:618893, End 618912
Start:618912, End 618931
Start:618931, End 618950
Start:618950, End 618968
Start:618968, End 618986
Start:618986, End 619005
Start:619005, End 619024
Start:619024, End 619043
Start:619043, End 619062
Start:619062, End 619080
Start:619080, End 619099
Start:619099, End 619112
Start:619112, End 619131
Start:619131, End 619150
Start:619150, End 619169
Start:619169, End 619188
Start:619188, End 619206
Start:619206, End 619225
Start:619225, End 619243
Start:619243, End 619264
Start:619264, End 619283
Start:619283, End 619297
Start:619297, End 619316
Start:619316, End 619334
Start:619334, End 619353
Start:619353, End 619367
Start:619367, End 619383
Start:619383, End 619402
Start:619402, End 619421
Start:619421, End 619441
Start:619441, End 619460
Start:619460, End 619480
Start:619480, End 619499


Start:630928, End 630949
Start:630949, End 630971
Start:630971, End 630990
Start:630990, End 631012
Start:631012, End 631028
Start:631028, End 631044
Start:631044, End 631063
Start:631063, End 631081
Start:631081, End 631100
Start:631100, End 631118
Start:631118, End 631137
Start:631137, End 631156
Start:631156, End 631178
Start:631178, End 631197
Start:631197, End 631216
Start:631216, End 631235
Start:631235, End 631252
Start:631252, End 631265
Start:631265, End 631287
Start:631287, End 631306
Start:631306, End 631320
Start:631320, End 631340
Start:631340, End 631359
Start:631359, End 631378
Start:631378, End 631397
Start:631397, End 631416
Start:631416, End 631435
Start:631435, End 631455
Start:631455, End 631473
Start:631473, End 631492
Start:631492, End 631511
Start:631511, End 631530
Start:631530, End 631543
Start:631543, End 631565
Start:631565, End 631576
Start:631576, End 631595
Start:631595, End 631610
Start:631610, End 631629
Start:631629, End 631645
Start:631645, End 631660


Start:648306, End 648325
Start:648325, End 648345
Start:648345, End 648363
Start:648363, End 648376
Start:648376, End 648395
Start:648395, End 648414
Start:648414, End 648432
Start:648432, End 648450
Start:648450, End 648469
Start:648469, End 648489
Start:648489, End 648508
Start:648508, End 648527
Start:648527, End 648541
Start:648541, End 648559
Start:648559, End 648578
Start:648578, End 648596
Start:648596, End 648618
Start:648618, End 648635
Start:648635, End 648654
Start:648654, End 648673
Start:648673, End 648686
Start:648686, End 648706
Start:648706, End 648725
Start:648725, End 648744
Start:648744, End 648755
Start:648755, End 648775
Start:648775, End 648794
Start:648794, End 648803
Start:648803, End 648822
Start:648822, End 648841
Start:648841, End 648860
Start:648860, End 648879
Start:648879, End 648894
Start:648894, End 648905
Start:648905, End 648922
Start:648922, End 648941
Start:648941, End 648957
Start:648957, End 648976
Start:648976, End 648995
Start:648995, End 649013


Start:665535, End 665554
Start:665554, End 665573
Start:665573, End 665591
Start:665591, End 665609
Start:665609, End 665628
Start:665628, End 665649
Start:665649, End 665667
Start:665667, End 665678
Start:665678, End 665690
Start:665690, End 665709
Start:665709, End 665728
Start:665728, End 665747
Start:665747, End 665765
Start:665765, End 665787
Start:665787, End 665807
Start:665807, End 665826
Start:665826, End 665845
Start:665845, End 665863
Start:665863, End 665882
Start:665882, End 665901
Start:665901, End 665920
Start:665920, End 665940
Start:665940, End 665959
Start:665959, End 665978
Start:665978, End 665997
Start:665997, End 666020
Start:666020, End 666038
Start:666038, End 666058
Start:666058, End 666077
Start:666077, End 666096
Start:666096, End 666115
Start:666115, End 666136
Start:666136, End 666155
Start:666155, End 666174
Start:666174, End 666194
Start:666194, End 666213
Start:666213, End 666232
Start:666232, End 666255
Start:666255, End 666275
Start:666275, End 666294


In [35]:
qio = np.array(query_item_order)
lbls = np.array(labels)
qio_test = np.array(query_item_order_test)
lbls_test = np.array(labels_test)
# np.loadtxt np.savetxt
path_to_save="D:\\Courses\\InfoStorage\\Project\\Data\\Numpy_Objects\\"
X.to_pickle(path_to_save+"AllFeaturesCombined.pkl")
Train.to_pickle(path_to_save+"Train.pkl")
Test.to_pickle(path_to_save+"Test.pkl")
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings8_train.txt',train,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings8AfterPairWise_Xp.txt',Xp,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings8AfterPairWise_yp.txt',yp,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings8_test.txt',test,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings8_query_item_order.txt',qio,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings8_labels.txt',lbls,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings8_query_item_order_test.txt',qio_test,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings8_labels_test.txt',lbls_test,fmt='%1.15f')

# All Features + Reduced Embeddings 6 

In [12]:
vector_gensimProduct6.head()

,itemId,vector_gensim_product
0,1,"[0.026879458, -0.0780781, -0.02600028, -0.0255..."
1,2,"[0.07616928, -0.06164205, -0.06643903, -0.0568..."
2,3,"[0.026013615, -0.018897766, 0.05609452, 0.0751..."
3,4,"[-0.061810836, 0.04790794, -0.0203071, -0.0134..."
4,6,"[0.072862014, 0.05055193, -0.015982823, -0.031..."


In [13]:
vector_gensimQuery6.head()

,queryId,vector_gensim_query
0,1,"[0.0034976015, 0.04624076, 0.046653748, 0.0678..."
1,2,"[0.06295683, -0.058646996, 0.07696536, -0.0637..."
2,3,"[0.013552657, 0.026750585, 0.07083171, 0.06842..."
3,4,"[0.039062634, -0.06316595, 0.07100032, 0.04891..."
4,5,"[-0.0021301818, -0.023087831, 0.062389884, -0...."


In [15]:
X = X.drop(['vector_gensim_query','vector_gensim_product'],axis=1)
X.head()

,queryId,itemId,Relevance,is.test,item_show_count,Item_clicks,Item_view_count,Item_purchase_count,userShow,userView,userPurchase,CTR,View Rate,Click Value Rate
0,1,7518,0,False,148.0,70.0,57.0,0.0,47.0,3.0,1.0,0.472973,0.385135,0.000000
1,1,71,0,False,192.0,92.0,73.0,2.0,102.0,2.0,1.0,0.479167,0.380208,0.010417
2,1,30311,0,False,108.0,164.0,116.0,2.0,37.0,2.0,1.0,1.518519,1.074074,0.018519
3,1,7837,0,False,281.0,220.0,191.0,3.0,82.0,5.0,1.0,0.782918,0.679715,0.010676
4,1,30792,0,False,389.0,475.0,445.0,11.0,143.0,11.0,1.0,1.221080,1.143959,0.028278


In [16]:
X = pd.merge(X,vector_gensimProduct6,on='itemId',how='left')
X = pd.merge(X,vector_gensimQuery6,on='queryId',how='left')

In [17]:
X.head()

,queryId,itemId,Relevance,is.test,item_show_count,Item_clicks,Item_view_count,Item_purchase_count,userShow,userView,userPurchase,CTR,View Rate,Click Value Rate,vector_gensim_product,vector_gensim_query
0,1,7518,0,False,148.0,70.0,57.0,0.0,47.0,3.0,1.0,0.472973,0.385135,0.000000,"[-0.019376937, 0.055629157, 0.04647832, 0.0143...","[0.0034976015, 0.04624076, 0.046653748, 0.0678..."
1,1,71,0,False,192.0,92.0,73.0,2.0,102.0,2.0,1.0,0.479167,0.380208,0.010417,"[0.029152757, 0.06757403, -0.0018385635, -0.02...","[0.0034976015, 0.04624076, 0.046653748, 0.0678..."
2,1,30311,0,False,108.0,164.0,116.0,2.0,37.0,2.0,1.0,1.518519,1.074074,0.018519,"[-0.03748395, 0.05580625, 0.05709914, -0.06844...","[0.0034976015, 0.04624076, 0.046653748, 0.0678..."
3,1,7837,0,False,281.0,220.0,191.0,3.0,82.0,5.0,1.0,0.782918,0.679715,0.010676,"[0.046882924, -0.036349002, 0.0012545191, 0.05...","[0.0034976015, 0.04624076, 0.046653748, 0.0678..."
4,1,30792,0,False,389.0,475.0,445.0,11.0,143.0,11.0,1.0,1.221080,1.143959,0.028278,"[-0.040539082, -0.05071903, -0.06422632, -0.08...","[0.0034976015, 0.04624076, 0.046653748, 0.0678..."


In [18]:
Train = X[X['is.test']==False]
Test = X[X['is.test']== True]
Train= Train.drop(['is.test'], axis=1)
Test= Test.drop(['is.test'], axis=1)

In [19]:
test=[]
query_item_order_test=[]
labels_test=[]
for index,row in Test.iterrows():
    r=[]
    query_item_order_test.append([row['queryId'],row['itemId']])
    
    r.append(row['item_show_count'])
    r.append(row['Item_clicks'])
    r.append(row['Item_view_count'])
    r.append(row['Item_purchase_count'])
    r.append(row['CTR'])
    r.append(row['userShow'])
    r.append(row['userView'])
    r.append(row['userPurchase'])
    r.append(row['View Rate'])
    r.append(row['Click Value Rate'])
    r= r + row['vector_gensim_query'].tolist()
    r= r + row['vector_gensim_product'].tolist()
    
    test.append(r)
    labels_test.append(row['Relevance'])

In [20]:
train=[]
query_item_order=[]
labels=[]
for index,row in Train.iterrows():
    r=[]
    query_item_order.append([row['queryId'],row['itemId']])
    
    r.append(row['item_show_count'])
    r.append(row['Item_clicks'])
    r.append(row['Item_view_count'])
    r.append(row['Item_purchase_count'])
    r.append(row['CTR'])
    r.append(row['userShow'])
    r.append(row['userView'])
    r.append(row['userPurchase'])
    r.append(row['View Rate'])
    r.append(row['Click Value Rate'])
    r= r + row['vector_gensim_query'].tolist()
    r= r + row['vector_gensim_product'].tolist()
    
    train.append(r)
    labels.append(row['Relevance'])

In [23]:
train = np.array(train)
test= np.array(test)
sc = StandardScaler()
train = sc.fit_transform(train)
test = sc.transform(test)
L=Train['queryId'].value_counts(sort=True, ascending=True)
L = dict(L)
L= OrderedDict(sorted(L.items()))
print(len(L))
print(len(labels))

35615
703519


In [24]:
#http://fa.bianp.net/blog/2012/learning-to-rank-with-scikit-learn-the-pairwise-transform/
k = 0
Xp, yp, diff = [], [], []
s=0
e=0
theta = np.deg2rad(60)
w = np.array([np.sin(theta), np.cos(theta)]) #for plotting
for key in L:
    comb = itertools.combinations(range(L.get(key)), 2)
    #create the first batch
    s=e
    e=s+L.get(key) -1
    print("Start:%d, End %d" %(s,e))
    for (i, j) in comb:
        i=i+s
        j=j+e-1
        if labels[i] == labels[j] or query_item_order[i][0] != query_item_order[j][0] :
            # skip if same target or different group(different qIds)
            continue
        Xp.append(train[i] - train[j])
        diff.append(labels[i] - labels[j])
        yp.append(np.sign(diff[-1]))
        # output balanced classes
        if yp[-1] != (-1) ** k:
            yp[-1] *= -1
            Xp[-1] *= -1
            diff[-1] *= -1
        k += 1
Xp, yp, diff = map(np.asanyarray, (Xp, yp, diff))

Start:0, End 19
Start:19, End 40
Start:40, End 59
Start:59, End 78
Start:78, End 96
Start:96, End 118
Start:118, End 136
Start:136, End 155
Start:155, End 174
Start:174, End 193
Start:193, End 212
Start:212, End 231
Start:231, End 250
Start:250, End 271
Start:271, End 289
Start:289, End 309
Start:309, End 330
Start:330, End 352
Start:352, End 371
Start:371, End 390
Start:390, End 409
Start:409, End 432
Start:432, End 451
Start:451, End 470
Start:470, End 489
Start:489, End 508
Start:508, End 527
Start:527, End 546
Start:546, End 565
Start:565, End 584
Start:584, End 607
Start:607, End 625
Start:625, End 646
Start:646, End 665
Start:665, End 684
Start:684, End 703
Start:703, End 722
Start:722, End 742
Start:742, End 762
Start:762, End 780
Start:780, End 801
Start:801, End 815
Start:815, End 834
Start:834, End 854
Start:854, End 872
Start:872, End 891
Start:891, End 913
Start:913, End 932
Start:932, End 952
Start:952, End 970
Start:970, End 989
Start:989, End 1008
Start:1008, End 1027
St

Start:15005, End 15025
Start:15025, End 15046
Start:15046, End 15065
Start:15065, End 15085
Start:15085, End 15104
Start:15104, End 15123
Start:15123, End 15138
Start:15138, End 15157
Start:15157, End 15177
Start:15177, End 15196
Start:15196, End 15215
Start:15215, End 15234
Start:15234, End 15253
Start:15253, End 15272
Start:15272, End 15291
Start:15291, End 15310
Start:15310, End 15332
Start:15332, End 15351
Start:15351, End 15370
Start:15370, End 15385
Start:15385, End 15404
Start:15404, End 15421
Start:15421, End 15435
Start:15435, End 15445
Start:15445, End 15463
Start:15463, End 15482
Start:15482, End 15501
Start:15501, End 15520
Start:15520, End 15539
Start:15539, End 15550
Start:15550, End 15569
Start:15569, End 15579
Start:15579, End 15596
Start:15596, End 15615
Start:15615, End 15634
Start:15634, End 15655
Start:15655, End 15674
Start:15674, End 15693
Start:15693, End 15713
Start:15713, End 15733
Start:15733, End 15752
Start:15752, End 15771
Start:15771, End 15790
Start:15790

Start:27957, End 27970
Start:27970, End 27992
Start:27992, End 28011
Start:28011, End 28028
Start:28028, End 28042
Start:28042, End 28061
Start:28061, End 28079
Start:28079, End 28098
Start:28098, End 28116
Start:28116, End 28136
Start:28136, End 28155
Start:28155, End 28174
Start:28174, End 28197
Start:28197, End 28216
Start:28216, End 28235
Start:28235, End 28254
Start:28254, End 28270
Start:28270, End 28289
Start:28289, End 28308
Start:28308, End 28318
Start:28318, End 28337
Start:28337, End 28356
Start:28356, End 28374
Start:28374, End 28393
Start:28393, End 28412
Start:28412, End 28431
Start:28431, End 28451
Start:28451, End 28470
Start:28470, End 28488
Start:28488, End 28507
Start:28507, End 28526
Start:28526, End 28544
Start:28544, End 28564
Start:28564, End 28583
Start:28583, End 28602
Start:28602, End 28621
Start:28621, End 28640
Start:28640, End 28659
Start:28659, End 28678
Start:28678, End 28696
Start:28696, End 28715
Start:28715, End 28732
Start:28732, End 28751
Start:28751

Start:40607, End 40626
Start:40626, End 40640
Start:40640, End 40659
Start:40659, End 40678
Start:40678, End 40697
Start:40697, End 40716
Start:40716, End 40735
Start:40735, End 40754
Start:40754, End 40773
Start:40773, End 40792
Start:40792, End 40818
Start:40818, End 40836
Start:40836, End 40854
Start:40854, End 40872
Start:40872, End 40891
Start:40891, End 40919
Start:40919, End 40939
Start:40939, End 40958
Start:40958, End 40977
Start:40977, End 40997
Start:40997, End 41019
Start:41019, End 41039
Start:41039, End 41058
Start:41058, End 41078
Start:41078, End 41097
Start:41097, End 41116
Start:41116, End 41135
Start:41135, End 41155
Start:41155, End 41175
Start:41175, End 41195
Start:41195, End 41214
Start:41214, End 41233
Start:41233, End 41252
Start:41252, End 41271
Start:41271, End 41290
Start:41290, End 41309
Start:41309, End 41328
Start:41328, End 41347
Start:41347, End 41366
Start:41366, End 41385
Start:41385, End 41403
Start:41403, End 41422
Start:41422, End 41441
Start:41441

Start:52415, End 52434
Start:52434, End 52446
Start:52446, End 52465
Start:52465, End 52482
Start:52482, End 52501
Start:52501, End 52520
Start:52520, End 52539
Start:52539, End 52559
Start:52559, End 52578
Start:52578, End 52594
Start:52594, End 52613
Start:52613, End 52632
Start:52632, End 52651
Start:52651, End 52675
Start:52675, End 52694
Start:52694, End 52713
Start:52713, End 52732
Start:52732, End 52748
Start:52748, End 52764
Start:52764, End 52776
Start:52776, End 52788
Start:52788, End 52807
Start:52807, End 52828
Start:52828, End 52847
Start:52847, End 52865
Start:52865, End 52883
Start:52883, End 52897
Start:52897, End 52916
Start:52916, End 52935
Start:52935, End 52954
Start:52954, End 52973
Start:52973, End 52992
Start:52992, End 53010
Start:53010, End 53027
Start:53027, End 53060
Start:53060, End 53079
Start:53079, End 53097
Start:53097, End 53116
Start:53116, End 53135
Start:53135, End 53149
Start:53149, End 53168
Start:53168, End 53188
Start:53188, End 53203
Start:53203

...

Start:605237, End 605256
Start:605256, End 605275
Start:605275, End 605294
Start:605294, End 605314
Start:605314, End 605333
Start:605333, End 605352
Start:605352, End 605371
Start:605371, End 605390
Start:605390, End 605402
Start:605402, End 605421
Start:605421, End 605440
Start:605440, End 605459
Start:605459, End 605478
Start:605478, End 605497
Start:605497, End 605516
Start:605516, End 605537
Start:605537, End 605556
Start:605556, End 605569
Start:605569, End 605588
Start:605588, End 605609
Start:605609, End 605628
Start:605628, End 605647
Start:605647, End 605669
Start:605669, End 605688
Start:605688, End 605707
Start:605707, End 605725
Start:605725, End 605744
Start:605744, End 605764
Start:605764, End 605781
Start:605781, End 605800
Start:605800, End 605820
Start:605820, End 605833
Start:605833, End 605852
Start:605852, End 605871
Start:605871, End 605891
Start:605891, End 605910
Start:605910, End 605929
Start:605929, End 605947
Start:605947, End 605965
Start:605965, End 605984


Start:619932, End 619951
Start:619951, End 619970
Start:619970, End 619989
Start:619989, End 620009
Start:620009, End 620028
Start:620028, End 620049
Start:620049, End 620068
Start:620068, End 620077
Start:620077, End 620096
Start:620096, End 620115
Start:620115, End 620133
Start:620133, End 620151
Start:620151, End 620170
Start:620170, End 620189
Start:620189, End 620208
Start:620208, End 620227
Start:620227, End 620246
Start:620246, End 620265
Start:620265, End 620286
Start:620286, End 620306
Start:620306, End 620325
Start:620325, End 620344
Start:620344, End 620358
Start:620358, End 620371
Start:620371, End 620380
Start:620380, End 620399
Start:620399, End 620418
Start:620418, End 620437
Start:620437, End 620456
Start:620456, End 620475
Start:620475, End 620494
Start:620494, End 620515
Start:620515, End 620534
Start:620534, End 620553
Start:620553, End 620573
Start:620573, End 620592
Start:620592, End 620611
Start:620611, End 620632
Start:620632, End 620651
Start:620651, End 620670


Start:633663, End 633682
Start:633682, End 633699
Start:633699, End 633718
Start:633718, End 633737
Start:633737, End 633756
Start:633756, End 633775
Start:633775, End 633795
Start:633795, End 633814
Start:633814, End 633833
Start:633833, End 633853
Start:633853, End 633872
Start:633872, End 633891
Start:633891, End 633910
Start:633910, End 633923
Start:633923, End 633944
Start:633944, End 633963
Start:633963, End 633982
Start:633982, End 634001
Start:634001, End 634020
Start:634020, End 634039
Start:634039, End 634057
Start:634057, End 634070
Start:634070, End 634089
Start:634089, End 634107
Start:634107, End 634126
Start:634126, End 634143
Start:634143, End 634162
Start:634162, End 634180
Start:634180, End 634192
Start:634192, End 634212
Start:634212, End 634231
Start:634231, End 634250
Start:634250, End 634269
Start:634269, End 634288
Start:634288, End 634302
Start:634302, End 634321
Start:634321, End 634340
Start:634340, End 634359
Start:634359, End 634378
Start:634378, End 634388


Start:648102, End 648122
Start:648122, End 648141
Start:648141, End 648161
Start:648161, End 648180
Start:648180, End 648199
Start:648199, End 648219
Start:648219, End 648228
Start:648228, End 648249
Start:648249, End 648269
Start:648269, End 648287
Start:648287, End 648306
Start:648306, End 648325
Start:648325, End 648345
Start:648345, End 648363
Start:648363, End 648376
Start:648376, End 648395
Start:648395, End 648414
Start:648414, End 648432
Start:648432, End 648450
Start:648450, End 648469
Start:648469, End 648489
Start:648489, End 648508
Start:648508, End 648527
Start:648527, End 648541
Start:648541, End 648559
Start:648559, End 648578
Start:648578, End 648596
Start:648596, End 648618
Start:648618, End 648635
Start:648635, End 648654
Start:648654, End 648673
Start:648673, End 648686
Start:648686, End 648706
Start:648706, End 648725
Start:648725, End 648744
Start:648744, End 648755
Start:648755, End 648775
Start:648775, End 648794
Start:648794, End 648803
Start:648803, End 648822


Start:661569, End 661588
Start:661588, End 661607
Start:661607, End 661625
Start:661625, End 661644
Start:661644, End 661663
Start:661663, End 661684
Start:661684, End 661696
Start:661696, End 661715
Start:661715, End 661732
Start:661732, End 661751
Start:661751, End 661770
Start:661770, End 661789
Start:661789, End 661808
Start:661808, End 661827
Start:661827, End 661847
Start:661847, End 661874
Start:661874, End 661891
Start:661891, End 661910
Start:661910, End 661928
Start:661928, End 661947
Start:661947, End 661963
Start:661963, End 661982
Start:661982, End 662002
Start:662002, End 662021
Start:662021, End 662041
Start:662041, End 662052
Start:662052, End 662069
Start:662069, End 662089
Start:662089, End 662108
Start:662108, End 662127
Start:662127, End 662144
Start:662144, End 662164
Start:662164, End 662183
Start:662183, End 662200
Start:662200, End 662219
Start:662219, End 662236
Start:662236, End 662255
Start:662255, End 662274
Start:662274, End 662293
Start:662293, End 662310


In [25]:
qio = np.array(query_item_order)
lbls = np.array(labels)
qio_test = np.array(query_item_order_test)
lbls_test = np.array(labels_test)
# np.loadtxt np.savetxt
path_to_save="D:\\Courses\\InfoStorage\\Project\\Data\\Numpy_Objects\\"
X.to_pickle(path_to_save+"AllFeaturesCombined.pkl")
Train.to_pickle(path_to_save+"Train.pkl")
Test.to_pickle(path_to_save+"Test.pkl")
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings6_train.txt',train,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings6AfterPairWise_Xp.txt',Xp,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings6AfterPairWise_yp.txt',yp,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings6_test.txt',test,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings6_query_item_order.txt',qio,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings6_labels.txt',lbls,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings6_query_item_order_test.txt',qio_test,fmt='%1.15f')
np.savetxt(path_to_save+'allFeaturesWithWord2VecEmbeddings6_labels_test.txt',lbls_test,fmt='%1.15f')